# 텍스트 데이터로 MBTI 예측하기
- 과제(task) : Text Classification
- 데이터셋 : MBTI 500 [kaggle](https://www.kaggle.com/mercurio117/mbti-500/data)
  - 전처리된 텍스트 데이터와 MBTI 유형으로 이루어짐
- 주요 참고 코드 : [MBTI 500 - 84% Accuracy](https://www.kaggle.com/clebermarques/mbti-500-84-accuracy)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
import pickle
import os.path
import plotly.offline as pyo
import plotly.graph_objs as go
import spacy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

## 1. 데이터 로드 및 확인
- kaggle에서 데이터 다운로드 후 `read_csv` 사용하여 데이터 로드
- 훈련 데이터 : 74357개, 테스트 데이터 : 9337개
- 훈련 데이터에만 MBTI `type` 컬럼 존재
- 다른 MBTI 데이터셋([(MBTI) Myers-Briggs Personality Type Dataset](https://www.kaggle.com/datasnaek/mbti-type)) 과의 차이점 
  - 데이터 크기가 크기 때문에 모델링 시에 연산량을 신경써야함
  - 텍스트 데이터가 이미 전처리(tokenization / Stemming 또는 Lemmatization)가 되어 있음


In [ ]:
data_dir = './dataset/'

train = pd.read_csv(os.path.join(data_dir, 'MBTI_train.csv'), encoding='ISO 8859-1', header=None, names=['type', 'posts'])
test = pd.read_csv(os.path.join(data_dir, 'MBTI_test.csv'), encoding='ISO 8859-1', header=None, names=['posts'])

print(train.shape, test.shape) # 74357, 9337
train.head()

(74357, 2) (9337, 1)


,type,posts
0,INTP,say process model list like subscriber channel...
1,INFJ,upon much manipulate retail finish like sacrif...
2,INFJ,fit yes certain bff social feel goal go know n...
3,INTJ,complete love within someone ideal joke solvea...
4,ENTJ,public strictly thing person x question person...


In [ ]:
test.head()

,posts
0,get accept ya bite well stop important open lo...
1,offer rebel something war people friend block ...
2,soulmates contradiction easy basic recurrence ...
3,run nature q test let sound sarcastically irri...
4,hour fast fast suspend see strict wampum eight...


## 2. 모델 로드 또는 재생성 후 학습
- 이미 저장된 모델이 있는 경우 해당 모델을 불러오고 그렇지 않다면 재생성하여 새로 학습시킴

In [ ]:
# 이미 만들어진 모델이 있어서 재생성해야하는지 여부를 지정
recreate_model=False

In [ ]:
# 해당 이름의 모델 파일이 있다면 모델 학습을 수행하지 않음
filename = 'mbti_svm_v2.sav'

In [ ]:
# 만약 모델이 존재하지 않는다면 모델을 재생성
if not os.path.isfile(filename):
    recreate_model=True

In [ ]:
X = train['posts'] # features
y = train['type']  # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 모델 재생성 여부 확인
if recreate_model:    
    
    # vectorizer 정의 및 fit_transform
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    
    # 훈련
    clf = LinearSVC()
    clf.fit(X_train_tfidf, y_train)
    
    # vectorizer 및 모델 파이프라인
    text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
    text_clf.fit(X_train, y_train)
    
    # 모델 저장
    pickle.dump(text_clf, open(filename, 'wb'))

# 모델 재생성하지 않으면 기존 저장된 모델 불러오기
else:
    # loading the model from disk
    text_clf = pickle.load(open(filename, 'rb'))

In [ ]:
predictions = text_clf.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

        ENFJ       0.83      0.56      0.67       210
        ENFP       0.77      0.70      0.73       828
        ENTJ       0.87      0.71      0.78       430
        ENTP       0.81      0.80      0.80      1582
        ESFJ       0.80      0.30      0.43        27
        ESFP       0.71      0.43      0.54        51
        ESTJ       0.00      0.00      0.00        11
        ESTP       0.89      0.80      0.84       169
        INFJ       0.78      0.82      0.80      2232
        INFP       0.77      0.77      0.77      1720
        INTJ       0.78      0.83      0.81      3225
        INTP       0.79      0.84      0.82      3493
        ISFJ       0.69      0.47      0.56        92
        ISFP       0.75      0.45      0.56       139
        ISTJ       0.81      0.47      0.59       198
        ISTP       0.84      0.72      0.77       465

    accuracy                           0.79     14872
   macro avg       0.74   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
print(f"Overall accuracy of the model: {round(metrics.accuracy_score(y_test, predictions),10)}")

Overall accuracy of the model: 0.7893356643


In [ ]:
predictions = text_clf.predict(test['posts'])

In [ ]:
print(predictions)

['ENFP' 'ENTP' 'INTJ' ... 'INTP' 'ENFP' 'INFP']


In [ ]:
# submission
sub_dir = './sub'
if not os.path.exists(sub_dir):
  os.mkdir(sub_dir)

sub = pd.DataFrame(predictions)
assert sub.shape == test.shape
sub.to_csv(os.path.join(sub_dir, 'baseline.csv'), index=False, header=None) # no header
sub.head()

,0
0,ENFP
1,ENTP
2,INTJ
3,INTJ
4,INTJ
